<a href="https://colab.research.google.com/github/Kaggle-challange-group8/Titanic_space_ship_group_8/blob/main/Kaggle_challange_group8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
test_datapath ='https://raw.githubusercontent.com/Kaggle-challange-group8/Titanic_space_ship_group_8/refs/heads/main/test-3.csv'
test_df = pd.read_csv(test_datapath)
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [4]:
train_dataset='https://raw.githubusercontent.com/Kaggle-challange-group8/Titanic_space_ship_group_8/refs/heads/main/train-3.csv'
train_df=pd.read_csv(train_dataset)
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, BatchNormalization

# Load data
train_df = pd.read_csv('/content/train-3.csv')
test_df = pd.read_csv('/content/test-3.csv')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Set random seed
np.random.seed(42)
tf.random.set_seed(42)

# Load data
df = pd.read_csv("/content/train-3.csv")
test_df = pd.read_csv("/content/test-3.csv")

# Save PassengerId for submission
test_passenger_ids = test_df['PassengerId']

# Drop unused columns
drop_cols = ['PassengerId', 'Name', 'Cabin']
df.drop(columns=drop_cols, inplace=True, errors='ignore')
test_df.drop(columns=drop_cols, inplace=True, errors='ignore')

# Target
y = df['Transported'].astype(int)
df.drop(columns=['Transported'], inplace=True)

# Identify column types
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Fill missing values
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
df[categorical_cols] = df[categorical_cols].apply(lambda col: col.fillna(col.mode()[0]))

test_df[numeric_cols] = test_df[numeric_cols].fillna(df[numeric_cols].median())
test_df[categorical_cols] = test_df[categorical_cols].apply(lambda col: col.fillna(col.mode()[0]))

# One-hot encode
df = pd.get_dummies(df, columns=categorical_cols)
test_df = pd.get_dummies(test_df, columns=categorical_cols)

# Align columns
df, test_df = df.align(test_df, join='left', axis=1, fill_value=0)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)
X_scaled = np.nan_to_num(X_scaled)
X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

X_test = scaler.transform(test_df)
X_test = np.nan_to_num(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Split train/val
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Build model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),

    Conv1D(32, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=20,
          batch_size=32,
          callbacks=[early_stop],
          verbose=1)

# Evaluate
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"✅ Validation Accuracy: {val_acc:.4f}")

# Predict on test set
y_pred = model.predict(X_test)
y_pred_bool = (y_pred > 0.5).astype(bool).flatten()

# Save submission
submission = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Transported': y_pred_bool
})

submission.to_csv("submission.csv", index=False)
print("✅ Submission file saved as 'submission.csv'")


/tmp/ipython-input-4-4278866607.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[categorical_cols] = df[categorical_cols].apply(lambda col: col.fillna(col.mode()[0]))
/tmp/ipython-input-4-4278866607.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df[categorical_cols] = test_df[categorical_cols].apply(lambda col: col.fillna(col.mode()[0]))
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer u

Epoch 1/20
218/218 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.6908 - loss: 0.5915 - val_accuracy: 0.7079 - val_loss: 0.5817
Epoch 2/20
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7569 - loss: 0.4968 - val_accuracy: 0.7671 - val_loss: 0.4583
Epoch 3/20
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7704 - loss: 0.4816 - val_accuracy: 0.7780 - val_loss: 0.4350
Epoch 4/20
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7766 - loss: 0.4749 - val_accuracy: 0.7815 - val_loss: 0.4321
Epoch 5/20
218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7771 - loss: 0.4679 - val_accuracy: 0.7890 - val_loss: 0.4280
Epoch 6/20
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7776 - loss: 0.4622 - val_accuracy: 0.7890 - val_loss: 0.4307
Epoch 7/20
218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7800 - loss: 0.4612 - val_accuracy: 0.7918 - val_loss: 0.4292
Epoch 8/20
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7758 - loss: 0.4594 - val_accuracy